In [1]:
from collections import namedtuple
from pathlib import Path, PosixPath

import pandas as pd
from sqlalchemy import create_engine

from src import COMPLETED

In [2]:
PRICE_COLS = ['cms_certification_num', 'payer', 'code', 'internal_revenue_code',
              #'units',
              'description', 'inpatient_outpatient', 'price', 'code_disambiguator']

PK_COLS = ['cms_certification_num', 'code', 'inpatient_outpatient', 'internal_revenue_code',
           'code_disambiguator', 'payer']

HOMEPAGE_URL = "https://www.providence.org/"
CHARGEMASTER_URL = "https://www.providence.org/obp/wa/pricing-transparency"

CURRENT_DIR = Path().cwd()
TMP_DIR = CURRENT_DIR / 'tmp'
DATA_DIR = CURRENT_DIR / 'data'
PRICES_DIR = CURRENT_DIR / 'prices'

In [3]:
def load_sql_or_csv(fname: PosixPath) -> pd.DataFrame:
    """Either load saved csv or load from database and save as csv"""
    if fname.exists():
        print('loading csv')
        return pd.read_csv(fname, dtype={'cms_certification_num': str})
    else:
        print('loading from database')
        engine = create_engine("mysql+pymysql://root@localhost/hospital_price_transparency_v3")
        remaining_wa = pd.read_sql("select * from hospitals WHERE last_edited_by_username IS NULL AND state = 'WA'", engine)
        remaining_wa['cms_certification_num'] = remaining_wa['cms_certification_num'].astype(str)
        remaining_wa.to_csv('remaining_wa.csv', index=False)

        return remaining_wa

def create_hospital_row(df: pd.DataFrame, cms: str, homepage_url: str, charge_url: str) -> pd.DataFrame:
    t = df[df['cms_certification_num'] == cms].copy()
    t['homepage_url'] = homepage_url
    t['chargemaster_url'] = charge_url
    t['last_edited_by_username'] = 'joeeoj'

    return t

def parse_cms_from_fname(fname: PosixPath) -> str:
    s = fname.name.split('_')[0] if '_' in fname.name else fname.name.split('-')[0]
    return s.replace('~$', '')

def parse_location(fname: PosixPath) -> str:
    s = fname.name.split('_')[1] if '_' in fname.name else fname.name.split('-')[1]
    for v in ['providenceregionalmedicalcenter', 'regionalmedicalcenter', 'providence', 'medicalcenter']:
        s = s.replace(v, '')
    return s

def add_prices(prices: pd.Series) -> float:
    """Convert to cents, sum, then convert back to float"""
    cents = [0 if pd.isnull(p) else int(p) * 100 for p in prices]
    return sum(cents)/100.0

In [4]:
remaining_wa = load_sql_or_csv(Path('remaining_wa.csv'))
remaining_wa = remaining_wa[~remaining_wa.isin(COMPLETED)]

remaining_wa.sort_values('city').head(2)

loading csv


,cms_certification_num,name,address,city,state,zip5,beds,phone_number,homepage_url,chargemaster_url,last_edited_by_username
0,500031,GRAYS HARBOR COMMUNITY HOSPITAL,915 ANDERSON DRIVE,ABERDEEN,WA,98520,70,3605328330,NaN,NaN,NaN
1,500007,ISLAND HOSPITAL,1211 24TH STREET,ANACORTES,WA,98221,43,3602991300,NaN,NaN,NaN


## hospital

In [5]:
File = namedtuple('File', ['fname', 'cms', 'location'])

files = []
for f in DATA_DIR.glob('*.xlsx'):
    files.append(File(f, parse_cms_from_fname(f), parse_location(f)))

hospitals = []
for f in files:
    hospitals.append(create_hospital_row(remaining_wa, f.cms, HOMEPAGE_URL, CHARGEMASTER_URL))

hospitals = pd.concat(hospitals)
print(len(hospitals))

hospitals.sort_values(['name', 'cms_certification_num'])

9


,cms_certification_num,name,address,city,state,zip5,beds,phone_number,homepage_url,chargemaster_url,last_edited_by_username
57,500058,KADLEC REGIONAL MEDICAL CENTER,888 SWIFT BLVD,RICHLAND,WA,99352,141,5099464611,https://www.providence.org/,https://www.providence.org/obp/wa/pricing-tran...,joeeoj
71,500054,PROV SACRED HRT MED CTR & CHILDS HOSP.,101 WEST 8TH AVENUE,SPOKANE,WA,99204,615,5094743040,https://www.providence.org/,https://www.providence.org/obp/wa/pricing-tran...,joeeoj
9,500019,PROVIDENCE CENTRALIA HOSPITAL,914 S SCHEUBER ROAD,CENTRALIA,WA,98531,191,3607362803,https://www.providence.org/,https://www.providence.org/obp/wa/pricing-tran...,joeeoj
72,500077,PROVIDENCE HOLY FAMILY HOSPITAL,5633 NORTH LIDGERWOOD,SPOKANE,WA,99208,272,5094822450,https://www.providence.org/,https://www.providence.org/obp/wa/pricing-tran...,joeeoj
14,501326,PROVIDENCE MOUNT CARMEL HOSPITAL,982 EAST COLUMBIA,COLVILLE,WA,99114,25,5096842561,https://www.providence.org/,https://www.providence.org/obp/wa/pricing-tran...,joeeoj
24,500014,PROVIDENCE REGIONAL MEDICAL CENTER EVERETT,1321 COLBY AVENUE,EVERETT,WA,98201,268,4252612000,https://www.providence.org/,https://www.providence.org/obp/wa/pricing-tran...,joeeoj
11,501309,PROVIDENCE ST JOSEPH HOSPITAL,500 EAST WEBSTER,CHEWELAH,WA,99109,65,5099358211,https://www.providence.org/,https://www.providence.org/obp/wa/pricing-tran...,joeeoj
80,500002,PROVIDENCE ST MARY MEDICAL CENTER,401 W POPLAR ST,WALLA WALLA,WA,99362,146,5095225900,https://www.providence.org/,https://www.providence.org/obp/wa/pricing-tran...,joeeoj
45,500024,PROVIDENCE ST PETER HOSPITAL,413 LILLY ROAD NE,OLYMPIA,WA,98506,340,3604919480,https://www.providence.org/,https://www.providence.org/obp/wa/pricing-tran...,joeeoj


In [6]:
hospitals.to_csv('hospitals.csv', index=False)

## prices

In [7]:
# cols are slightly different names in the different sheets but in the same order
COLS = ['internal_revenue_code', 'description', 'code', 'unit_price', 'base_price']

In [8]:
def open_or_load_parquet(fname: PosixPath) -> pd.DataFrame:
    """Open saved parquet, otherwise open given fname and save as parquet.
    Basically speed up reloads because reading Excel files is slow."""
    fout = TMP_DIR / fname.with_suffix('.parquet').name

    if not fout.exists():
        gross = pd.read_excel(fname, sheet_name='Gross Charges', skiprows=[0,1,2,3])
        gross = gross.rename(columns={k: v for k,v in zip(gross.columns[:5], COLS)})
        gross['payer'] = 'GROSS CHARGE'
    
        cash = pd.read_excel(fname, sheet_name='Discount Cash Price - Gross', skiprows=[0,1,2,3])
        cash = cash.rename(columns={k: v for k,v in zip(cash.columns[:5], COLS)})
        cash['payer'] = 'CASH PRICE'

        output_cols = COLS + ['payer']
        df = pd.concat([gross[output_cols], cash[output_cols]])  # only save subset of cols with payer
        
        # check float type
        float_cols = ['unit_price', 'base_price']
        for col in float_cols:
            dtype = str(df[col].dtype)     
            if dtype in ['object', 'str']:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        df.to_parquet(fout)

    return pd.read_parquet(fout)

In [9]:
def parse_hospital(f: File) -> pd.DataFrame:
    df = open_or_load_parquet(f.fname)

    float_cols = ['unit_price', 'base_price']
    for col in float_cols:
        df[col] = df[col].round(2)  # i hate it but it works

    df['both_total'] = df[float_cols].apply(add_prices, axis=1).round(2)

    df = (df[['internal_revenue_code', 'description', 'code', 'payer', 'both_total']]
          .rename(columns={'both_total': 'price'}))
    df['inpatient_outpatient'] = 'BOTH'

    df['cms_certification_num'] = f.cms
    df['code_disambiguator'] = 'NONE'
    df['description'] = df['description'].str.strip()

    print(f'Dropping {df.duplicated(keep=False).sum():,} fully duplicated rows')
    df = df.drop_duplicates()

    # drop where price is null
    print(f"Dropping {df['price'].isnull().sum():,} rows where price is null")
    df = df.dropna(subset='price')

    # same for $0
    zero_dollars = (df['price'] == 0)
    print(f"Dropping {zero_dollars.sum():,} rows where price is $0")
    df = df[~zero_dollars]

    # fill in blank codes with NONE
    assert df['code'].isnull().sum() > 0
    df['code'] = df['code'].fillna('NONE')

    # dupe checks
    assert df.duplicated().sum() == 0
    assert df[PK_COLS].duplicated().sum() == 0

    # other checks
    assert df['price'].isnull().sum() == 0
    assert (df['price'] == 0).sum() == 0
    assert df['internal_revenue_code'].isnull().sum() == 0

    print(f"min price: {df['price'].min():,.2f}")
    print(f"max price: {df['price'].max():,.2f}")

    print(f'Total rows: {len(df):,}')

    return df

In [10]:
for f in files:
    print(f.fname)
    fout = PRICES_DIR / f'prices_{f.location}.csv'

    df = parse_hospital(f)
    df.to_csv(fout, index=False)

    print('-' * 80)

/Users/myu/repos/hospital-price-transparency-v3/src/providence/data/501326_providencemountcarmel_standardcharges.xlsx
Dropping 0 fully duplicated rows
Dropping 0 rows where price is null
Dropping 326 rows where price is $0
min price: 4.00
max price: 11,278.00
Total rows: 7,360
--------------------------------------------------------------------------------
/Users/myu/repos/hospital-price-transparency-v3/src/providence/data/500014_providenceregionalmedicalcentereverett_standardcharges.xlsx
Dropping 0 fully duplicated rows
Dropping 0 rows where price is null
Dropping 0 rows where price is $0
min price: 2.00
max price: 94,606.00
Total rows: 8,920
--------------------------------------------------------------------------------
/Users/myu/repos/hospital-price-transparency-v3/src/providence/data/500002_providencestmarymedicalcenter_standardcharges.xlsx
Dropping 0 fully duplicated rows
Dropping 0 rows where price is null
Dropping 0 rows where price is $0
min price: 2.00
max price: 64,968.00
T